<a href="https://colab.research.google.com/github/aditivin16/AI_Club_RLGames/blob/main/aditi_clip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
from transformers import CLIPModel, CLIPProcessor
from IPython.display import clear_output

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from collections import deque, namedtuple
import matplotlib.pyplot as plt
import numpy as np
import torch.distributions as distributions

In [ ]:
!pip install minigrid

In [ ]:
import minigrid
from minigrid.wrappers import ImgObsWrapper


pygame 2.5.2 (SDL 2.28.2, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
import gymnasium as gym

In [ ]:
env = gym.make("BabyAI-GoToRedBallGrey-v0", render_mode='rgb_array')
n_actions=7 #for this env

In [ ]:
# Creating the CLIP model
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch16")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16")
layer1 = torch.nn.Linear(1280, 100)
layer2 = torch.nn.Linear(100, 128)
relu = torch.nn.ReLU()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
def clip_model(text, image, direction):
    encoded = processor(text=text, images=image, return_tensors="pt", padding=True)
    clip_output = model(**encoded)
    direction_tensor = torch.tensor([direction])
    base_output = torch.cat((clip_output.text_model_output.pooler_output, clip_output.vision_model_output.pooler_output), dim=1)
    l1 = layer1(base_output)
    r = relu(l1)
    l2 = layer2(r)
    return l2
#here clip_output.text_model_output} will have information about the text encodings


In [ ]:
# Hyperparameters
actor_lr = 0.001
critic_lr = 0.001
gamma = 0.99
num_episodes = 100


In [ ]:
# Define the actor and critic networks
actor = nn.Sequential(
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.Linear(64, 64),
    nn.ReLU(),
    nn.Linear(64, 7),  # Adapted to the action space (7 actions in BabyAI)
)


critic = nn.Sequential(
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.Linear(64, 64),
    nn.ReLU(),
    nn.Linear(64, 1)
)


In [ ]:
# Defining the optimizer for actor and critic
optimizer_actor = optim.Adam(actor.parameters(), lr=actor_lr)
optimizer_critic = optim.Adam(critic.parameters(), lr=critic_lr)
rewards=[]

In [ ]:
from IPython.display import display, clear_output


In [ ]:
# Training loop
max_step=100
for episode in range(num_episodes):
    state = env.reset()
    net_reward = 0
    done = False
    trunc=False
    i=0
    step=0

    while not done and step<max_step and not trunc:
        step=step+1
        if(i==0):
          state_image = state[0]['image']  # Access the image
          state_image = torch.tensor(state_image, dtype=torch.float32)
          text_encoding = state[0]['mission']
          i=1
        else:
          state_image = state['image']  # Access the image
          state_image = torch.tensor(state_image, dtype=torch.float32)
          text_encoding = state['mission']



        # We give a text prompt of mission
        direction = torch.randint(0, 7, (1,))  # Randomly sample an action as a baseline
        encoding = clip_model(text_encoding, state_image, direction)
        action_probs = actor(encoding)
        action_distribution = distributions.Categorical(logits=action_probs)
        action = action_distribution.sample().item()
        # next_state, reward, done, info = env.step(action)
        print(action)
        next_state,reward,done,trunc,_= env.step(action)
        #print( reward, done, action, truc )
        state = next_state  # Update the state for the next iteration
        # Calculating the TD error
        state_value2 = critic(encoding)
        state_image2 = state['image']  # Access the image
        state_image2 = torch.tensor(state_image, dtype=torch.float32)
        text_encoding2 = state['mission']  # We give a text prompt of mission
        direction2 = torch.randint(0, 7, (1,))  # Randomly sample an action as a baseline
        encoding2 = clip_model(text_encoding2, state_image2, direction2)

        next_state_value = critic(encoding2)
        td_error = reward + gamma * (next_state_value - state_value2)

        # Updating the critic
        loss_critic = td_error.pow(2)
        optimizer_critic.zero_grad()
        # loss_critic.backward()
        # optimizer_critic.step()

        # Updating the actor using the TD error as a baseline
        log_prob = action_distribution.log_prob(torch.tensor(action))
        loss_actor = -log_prob * td_error.detach() + loss_critic
        optimizer_actor.zero_grad()
        loss_actor.backward()
        optimizer_actor.step()
        optimizer_critic.step()
        plt.imshow(env.render())
        plt.show()
        plt.pause(1e-3)
        clear_output()


        net_reward += reward

    print(f"Episode {episode + 1}/{num_episodes}, Net Reward: {net_reward}")

# Close the environment
env.close()


1
